In [1]:
# https://platform.olimpiada-ai.ro/problems/46

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [2]:
train = pd.read_csv("/kaggle/input/text-correction/train.csv")
test = pd.read_csv("/kaggle/input/text-correction/test.csv")

In [3]:
train['Text'] = 'Correct the sentence: ' + train['Text']
test['Text'] = 'Correct the sentence: ' + test['Text']

In [4]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-large").to(device)

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

2025-11-26 13:19:17.564230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764163157.734313      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764163157.786083      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
model.eval()

sentences = ['This is a wrong sentence', 'Is this one also?']

prompts = [f"Translate from English to French: {s}" for s in sentences]

inputs = tokenizer(prompts, truncation=True, padding="max_length", max_length=32, return_tensors='pt')
inputs = {k: v.to(device) for k, v in inputs.items()}

outputs = model.generate(**inputs)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

["C'est une mauvaise phrase", "Est-ce que c'est aussi?"]


In [7]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train)
train_ds = Dataset.train_test_split(train_ds, test_size=0.1, seed=42)


def preprocess(example):
    input_enc = tokenizer(example['Text'], padding="max_length", truncation=True, max_length=32, return_tensors='pt')
    input_enc = {k: v.to(device) for k, v in input_enc.items()}
    
    target_enc = tokenizer(example['RevisedText'], padding="max_length", truncation=True, max_length=32, return_tensors='pt')
    labels = target_enc['input_ids'].to(device)
    labels[labels==tokenizer.pad_token_id] = -100

    return {
        'input_ids': input_enc['input_ids'].squeeze(),
        'attention_mask': input_enc['attention_mask'].squeeze(),
        'labels': labels.squeeze()
    }

train_ds = train_ds.map(preprocess, remove_columns=train_ds.column_names['train'])

Map:   0%|          | 0/1452 [00:00<?, ? examples/s]

Map:   0%|          | 0/162 [00:00<?, ? examples/s]

In [8]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir='./result',
    report_to='none',
    do_train=True,
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=2e-5,
    weight_decay=1e-4,
    
    do_eval=True,
    per_device_eval_batch_size=1,
    eval_strategy='steps',
    save_strategy='epoch',
    eval_steps=0.0625
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds['train'],
    eval_dataset=train_ds['test'],
    processing_class=tokenizer
)

In [9]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
46,No log,0.323835
92,No log,0.251378
138,No log,0.244951
184,No log,0.227936
230,No log,0.223752
276,No log,0.207380
322,No log,0.206514
368,No log,0.205197
414,No log,0.207706
460,No log,0.197234


TrainOutput(global_step=726, training_loss=0.2885565534439297, metrics={'train_runtime': 1303.5014, 'train_samples_per_second': 1.114, 'train_steps_per_second': 0.557, 'total_flos': 196478042112000.0, 'train_loss': 0.2885565534439297, 'epoch': 1.0})

In [36]:
inputs = tokenizer(test['Text'].tolist(), padding="max_length", truncation=True, max_length=32, return_tensors='pt')
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = tokenizer.batch_decode(model.generate(**inputs), skip_special_tokens=True)

In [39]:
for i in range(5):
    print(f"Original: {test['Text'][i]}")
    print(f"Predicted: {outputs[i]}")
    print()

Original: Correct the sentence: He didn't eat no breakfast this morning.
Predicted: He didn't eat any breakfast this morning.

Original: Correct the sentence: 29. When I was young, I used to played soccer every day.
Predicted: 29. When I was young, I used to play soccer every day.

Original: Correct the sentence: The car is parked in front of the house.
Predicted: The car is parked in front of the house.

Original: Correct the sentence: They're house is beautiful, but it's location is far.
Predicted: Their house is beautiful, but its location is far.

Original: Correct the sentence: I can't believe she ate the whole cake herself!
Predicted: I can't believe she ate the whole cake herself!



In [41]:
subm = pd.DataFrame({
    'DatapointID': test['SampleID'],
    'RevisedText': outputs
})

subm

,DatapointID,RevisedText
0,1557,He didn't eat any breakfast this morning.
1,527,"29. When I was young, I used to play soccer ev..."
2,394,The car is parked in front of the house.
3,1790,"Their house is beautiful, but its location is ..."
4,434,I can't believe she ate the whole cake herself!
...,...,...
399,1113,This is the hardest puzzle I've ever solved.
400,694,She loves reading books by Jane Austen.
401,1495,The book is on the shelf.
402,922,Tim walked the dog in the park.


In [42]:
subm.to_csv("submission.csv", index=False)